In [ ]:
! pip install pandas

In [ ]:
! wget https://cloud.ynnk.dev/s/WZT8ia5MP93XEP7/download/example.csv

In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np

df = pd.read_csv("example.csv", engine='pyarrow', encoding="utf-8", delimiter=";")

In [ ]:
df = df.drop(
    ["BETRIEBSTAG", "BETREIBER_ID", "BETREIBER_ABK", "FAHRT_BEZEICHNER",
     "BETREIBER_NAME", "LINIEN_ID", "UMLAUF_ID", "VERKEHRSMITTEL_TEXT",
     "AB_PROGNOSE_STATUS", "AN_PROGNOSE_STATUS",
     "ZUSATZFAHRT_TF", "BPUIC", "DURCHFAHRT_TF"], axis=1)

In [ ]:
df["PRODUKT_ID"].unique()

In [ ]:
df = df[df["PRODUKT_ID"] == "Zug"]

In [ ]:
df = df[~df["FAELLT_AUS_TF"]]
df = df.drop(["FAELLT_AUS_TF", "PRODUKT_ID"], axis=1)

In [ ]:
df.dtypes

In [ ]:
df.replace('', np.nan, inplace=True)



In [ ]:
for time_col in ["ANKUNFTSZEIT", "ABFAHRTSZEIT"]:
  df[time_col] = pd.to_datetime(df[time_col], format="%d.%m.%Y %H:%M")

for time_col in ["AN_PROGNOSE", "AB_PROGNOSE"]:
  df[time_col] = pd.to_datetime(df[time_col], format="%d.%m.%Y %H:%M:%S")


In [ ]:
df = df[~(df["AN_PROGNOSE"].isna() ^ df["ANKUNFTSZEIT"].isna()) | ~(df["AB_PROGNOSE"].isna() ^ df["ABFAHRTSZEIT"].isna())]

In [ ]:
df["DELAY_ANKUFT"] = (df["AN_PROGNOSE"] - df["ANKUNFTSZEIT"]).astype('timedelta64[s]')
df["DELAY_ABFAHRT"] = (df["AB_PROGNOSE"] - df["ABFAHRTSZEIT"] ).astype('timedelta64[s]')

In [ ]:
df["DELAY_ANKUFT"] = df["DELAY_ANKUFT"].astype("Int64")
df["DELAY_ABFAHRT"] = df["DELAY_ABFAHRT"].astype("Int64")

In [ ]:
df = df[df["DELAY_ABFAHRT"].between(-300,300) | df["DELAY_ANKUFT"].between(-300,300)]

In [ ]:
df = df.dropna(subset=["HALTESTELLEN_NAME"])

In [ ]:
# Gibt die Züge aus die zu früh abgefahren sind :)
df[df["DELAY_ABFAHRT"] < 0].sort_values("DELAY_ABFAHRT")

In [ ]:
i = 0
for index, row in df.iterrows():
  if(pd.isnull(row["ABFAHRTSZEIT"])):
    i += 1
  df.loc[index,"FAHRT_ID"] = i
df["FAHRT_ID"] = df["FAHRT_ID"].astype(int)